In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from gold.gold_aggregate import add_year_column , aggregate_profit, 


spark = SparkSession.builder.appName("Aggregated_gold_table").getOrCreate()

df=spark.read.table("workspace.silver.enriched_joined_table")

df=add_year_column(df)
profit_year_cat_subcat_cust=aggregate_profit(df)

display(profit_year_cat_subcat_cust.head(10)) 

profit_year_cat_subcat_cust.write.format("delta").mode("overwrite").saveAsTable("workspace.gold.master")


year,category,sub_category,customer_id,total_profit
2014,Office Supplies,Paper,KB-16315,9.55
2014,Technology,Phones,RM-19675,173.02
2014,Office Supplies,Binders,LE-16810,-30.55
2014,Office Supplies,Binders,KL-16555,-77.45
2014,Office Supplies,Binders,GD-14590,5.0
2014,Office Supplies,Paper,SM-20005,5.23
2014,Technology,Accessories,DO-13435,-3.0
2014,Office Supplies,Storage,JM-16195,214.0
2014,Office Supplies,Art,MY-18295,1.5
2014,Office Supplies,Envelopes,PB-19105,204.07


In [0]:


df.createOrReplaceTempView(f"gold_aggregate_final")

profit_year=spark.sql(""" select year,round(sum(profit),2) as total_profit_year from gold_aggregate_final group by year order by year """)
profit_year.show()

+----+-----------------+
|year|total_profit_year|
+----+-----------------+
|2014|          40975.4|
|2015|         65706.25|
|2016|         68161.13|
|2017|        127175.23|
+----+-----------------+



In [0]:
profit_year_category=spark.sql(""" select year,category,round(sum(profit),2)as total_profit_year_category from gold_aggregate_final group by year,category order by year""")
profit_year_category.show()

+----+---------------+--------------------------+
|year|       category|total_profit_year_category|
+----+---------------+--------------------------+
|2014|      Furniture|                  -5174.65|
|2014|Office Supplies|                  22663.76|
|2014|     Technology|                  23486.29|
|2015|     Technology|                  36823.79|
|2015|      Furniture|                   3392.12|
|2015|Office Supplies|                  25490.34|
|2016|Office Supplies|                   35973.6|
|2016|     Technology|                  24437.38|
|2016|      Furniture|                   7750.15|
|2017|     Technology|                  78483.08|
|2017|Office Supplies|                  45330.39|
|2017|      Furniture|                   3361.76|
+----+---------------+--------------------------+



In [0]:
profit_customer=spark.sql(""" select customer_id,round(sum(profit),2) as total_profit_customer from gold_aggregate_final group by customer_id order by total_profit_customer desc""") 

profit_customer.show()

+-----------+---------------------+
|customer_id|total_profit_customer|
+-----------+---------------------+
|   FH-14275|             25850.07|
|   TC-20980|              8983.23|
|   RB-19360|              6976.36|
|   SC-20095|              5839.14|
|   HL-15040|              5611.46|
|   PR-18880|               5596.2|
|   AB-10105|              5483.75|
|   TA-21385|              4703.72|
|   CM-12385|              3912.03|
|   PS-19045|              3183.77|
|   KD-16495|              3030.05|
|   BM-11650|              2979.44|
|   DR-12940|              2869.08|
|   AR-10540|              2860.34|
|   TB-21400|              2821.71|
|   NM-18445|               2751.7|
|   SE-20110|              2661.23|
|   BS-11365|              2640.41|
|   HW-14935|              2635.84|
|   HM-14860|              2438.07|
+-----------+---------------------+
only showing top 20 rows


In [0]:
profit_customer_year=spark.sql(""" select customer_id,year,round(sum(profit),2) as total_profit_customer_year from gold_aggregate_final group by customer_id,year order by year""")
profit_customer_year.show()

+-----------+----+--------------------------+
|customer_id|year|total_profit_customer_year|
+-----------+----+--------------------------+
|   AC-10660|2014|                     -4.97|
|   TB-21595|2014|                      6.75|
|   JS-15595|2014|                   -204.14|
|   SV-20785|2014|                    189.13|
|   LA-16780|2014|                    399.29|
|   PB-19105|2014|                    204.07|
|   KB-16315|2014|                    -29.74|
|   BH-11710|2014|                    235.35|
|   DP-13000|2014|                     24.24|
|   MY-18295|2014|                   -103.13|
|   JD-16015|2014|                    -10.34|
|   CM-12445|2014|                     10.78|
|   SM-20005|2014|                     42.45|
|   KL-16555|2014|                      9.61|
|   LE-16810|2014|                    -12.83|
|   RM-19675|2014|                    200.12|
|   JM-16195|2014|                     273.4|
|   GD-14590|2014|                       5.0|
|   DO-13435|2014|                